In [1]:
import os
import gc
import sys
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

# Functions

In [14]:
def read_in_data():
    """
    """
    
    # Read in Features View
    df_ft = pd.read_csv("data/features.csv")
    pre_ = df_ft.shape[0]
    df_ft.drop_duplicates(inplace=True)
    assert df_ft.shape[0] == pre_
    
    # Read in Stores View
    df_stores = pd.read_csv("data/stores.csv")
    pre_ = df_stores.shape[0]
    df_stores.drop_duplicates(inplace=True)
    assert df_stores.shape[0] == pre_
    
    # Combine Stores and Features
    df_ft = pd.merge(
        df_ft,
        df_stores,
        how='left',
        on=['Store'],
        validate='m:1'
    )
    del df_stores
    gc.collect()
    
    return df_ft


def read_in_train_test():
    """
    """
    
    # Read in Features View
    train = pd.read_csv("data/train.csv")
    pre_ = train.shape[0]
    train.drop_duplicates(inplace=True)
    assert df_ft.shape[0] == pre_
    
    # Read in Stores View
    test = pd.read_csv("data/test.csv")
    pre_ = test.shape[0]
    test.drop_duplicates(inplace=True)
    assert test.shape[0] == pre_
    
    return train, test
    
    

# Read in and Prepare

In [27]:
# Fnames and Paths
fnames = os.listdir("data/")
print(fnames)

try:
    os.makedirs("output/")
except:
    pass
print("output directories made...")

['features.csv', 'sampleSubmission.csv', 'stores.csv', 'test.csv', 'train.csv']
output directories made...


Featurespace merged with store data...


In [106]:
# Feature Engineering
def add_season(data):
    """
    """
    
    # Date to Month
    data['Date'] = pd.to_datetime(data.Date)
    data['Month'] = data['Date'].dt.month.astype(int)
    
    # Season flags
    data['isSpring'] = (data['Month'].isin([3, 4, 5])).astype(int)
    data['isSummer'] = (data['Month'].isin([6, 7, 8])).astype(int)
    data['isFall'] = (data['Month'].isin([9, 10, 11])).astype(int)
    data['isWinter'] = (data['Month'].isin([12, 1, 2])).astype(int)
    
    return data

def add_cpi_cutoff(data):
    
    # Apply cutoff from EDA
    data['cpiUpperGroup'] = (data['CPI'] >= 75).astype(int)
    return data

def add_unemployment_cutoff(data):
    
    # Apply cutoff from EDA
    data['unemploymentAbove9'] = (data['Unemployment'] >= 9).astype(int)
    return data

def fill_cpi(data):
    
    # Get Type-specific CPI averages
    avgs = {typ: data.loc[data.Type==typ, :]['CPI'].mean() for typ in set(data['Type'])}
    data.loc[data.CPI.isnull(), 'CPI'] = data.Type.map(avgs)
    assert sum(data.CPI.isnull()) == 0
    return data

def fill_unemployment_typemonth(data):
    
    assert 'unemploymentAbove9' in data.columns
    
    # Get Type-Month specific Unemployments averages
    data['type_month'] = data.Type.astype(str)+"_"+data.Month.astype(str)
    avgs = data.groupby(by=['Type', 'Month'],
                       as_index=False).agg({'Unemployment': np.mean})
    avgs['type_month'] = avgs.Type.astype(str)+"_"+avgs.Month.astype(str)
    avgs = {tm: avgs.loc[avgs['type_month'] == tm, :]['Unemployment'].mean() for tm in set(avgs['type_month'])}
    data.loc[data.Unemployment.isnull(), 'Unemployment'] = data.type_month.map(avgs)
    data.drop(labels=['type_month'], axis=1, inplace=True)
    return data

def holiday_flag_to_bool(data):
    """
    """
    
    data.loc[:, 'IsHoliday'] = data['IsHoliday'].astype(int)
    return data

def flag_pre_holiday(data):
    """
    """
    
    # Sort
    data.sort_values(by=['Store', 'Month'], ascending=True, inplace=True)
    data.loc[:, 'upcoming_holiday'] = (data.IsHoliday.shift(-1)==1).astype(int)
    assert all(data.upcoming_holiday.notnull())
    
    return data

def temp_diff_from_store_month_mean(data):
    """
    Difference from store,month average temperature
    """
    
    # Get store,month mean reference table
    aggr = data.groupby(
        by=['Store', 'Month'], 
        as_index=False
    ).agg({'Temperature': np.mean})
    aggr.rename(columns={'Temperature': 'store_month_mean_temp'},
               inplace=True)
    
    # Merge back
    data = pd.merge(
        data,
        aggr,
        how='left',
        on=['Store', 'Month'],
        validate='m:1'
    )
    
    # Calculate Diff
    print(sorted(list(data.columns)))
    data.loc[:, 'temp_diff_store_month_mean'] = (
        data.Temperature - data.store_month_mean_temp
    )
    assert all(data.temp_diff_store_month_mean.notnull())
    
    return data

def temp_diff_from_lagged_max(data):
    """
    """
    # Sort
    data.sort_values(by=['Store', 'Date'],
                     ascending=True,
                     inplace=True)
    
    # Get rolling max
    data['store_temp_rlg_max'] = \
        data.groupby('Store')['Temperature'].rolling(window=4).max().reset_index(drop=True)
    
    mthly_max = data.groupby(by=['Store', 'Month'],
                             as_index=False).agg({'Temperature': 'max'})
    mthly_max['pair'] = mthly_max.Store.astype(str)+"_"+mthly_max.Month.astype(str)
    mthly_max = mthly_max.set_index('pair')['Temperature'].to_dict()
    
    # Backfill rolling max empty with Store Month max
    data.loc[:, 'pair'] = data.Store.astype(str)+"_"+data.Month.astype(str)
    data.loc[data['store_temp_rlg_max'].isnull(), 'store_temp_rlg_max'] = data.pair.map(mthly_max)
    assert all(data.store_temp_rlg_max.notnull())
    data.drop(labels=['pair'], axis=1, inplace=True)
    
    return data

def temp_cv_month(data):
    """
    """
    
    # SD
    data['temp_sigma'] = data.groupby(by=['Store', 'Month'])['Temperature'].std().reset_index(drop=True)
    # Mean
    data['temp_mean'] = data.groupby(by=['Store', 'Month'])['Temperature'].mean().reset_index(drop=True)
    # Ratio
    data['temperature_cv'] = (data.temp_sigma / data.temp_mean)
    data['temperature_cv'].fillna(0, inplace=True)
    assert all(data['temperature_cv'].notnull())
    data.drop(labels=['temp_sigma', 'temp_mean'], axis=1, inplace=True)
    
    return data

def fill_markdowns(data):
    
    # Replace negatives with 0
    for col in [x for x in data.columns if 'markdown' in x.lower()]:
        df.loc[df[col]<0, col] = 0
    
    # Fill Markdowns with KMEANS group assigned means
    
    
    

In [107]:
df_ft = read_in_data()
print("Featurespace merged with store data...")

Featurespace merged with store data...


In [108]:
# Feature Engineering
df_ft = add_season(df_ft)
df_ft = add_cpi_cutoff(df_ft)
df_ft = holiday_flag_to_bool(df_ft)
df_ft = add_unemployment_cutoff(df_ft)
df_ft = fill_cpi(df_ft)
df_ft = fill_unemployment_typemonth(df_ft)
df_ft = temp_diff_from_store_month_mean(df_ft)
df_ft = temp_diff_from_lagged_max(df_ft)
df_ft = temp_cv_month(df_ft)
df_ft = flag_pre_holiday(df_ft)

['CPI', 'Date', 'Fuel_Price', 'IsHoliday', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Month', 'Size', 'Store', 'Temperature', 'Type', 'Unemployment', 'cpiUpperGroup', 'isFall', 'isSpring', 'isSummer', 'isWinter', 'store_month_mean_temp', 'unemploymentAbove9']


In [109]:
df_ft.isnull().mean()

Store                         0.000000
Date                          0.000000
Temperature                   0.000000
Fuel_Price                    0.000000
MarkDown1                     0.507692
MarkDown2                     0.643346
MarkDown3                     0.558852
MarkDown4                     0.577045
MarkDown5                     0.505495
CPI                           0.000000
Unemployment                  0.000000
IsHoliday                     0.000000
Type                          0.000000
Size                          0.000000
Month                         0.000000
isSpring                      0.000000
isSummer                      0.000000
isFall                        0.000000
isWinter                      0.000000
cpiUpperGroup                 0.000000
unemploymentAbove9            0.000000
store_month_mean_temp         0.000000
temp_diff_store_month_mean    0.000000
store_temp_rlg_max            0.000000
temperature_cv                0.000000
upcoming_holiday         

0    7605
1     585
Name: IsHoliday, dtype: int64